# 1. 패키지 설치

In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [5]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [6]:
from langchain_pinecone import PineconeVectorStore

# 데이터를 처음 저장할 때
index_name = 'tax-upstage-index'

# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(document_list)
print(f"Chunked documents length: {len(chunked_documents)}")

# Initialize the PineconeVectorStore
database = PineconeVectorStore.from_documents(
    documents=[],  # Start with an empty list
    embedding=embedding,
    index_name=index_name
)

# Upload documents in batches
batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size: {batch_size}')
    batch = chunked_documents[i:i + batch_size]
    database.add_documents(batch)  # Add documents to the existing database

Chunked documents length: 371
index: 0, batch size: 100
index: 100, batch size: 100
index: 200, batch size: 100
index: 300, batch size: 100


# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [7]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [8]:
retrieved_docs

[Document(id='9b002c0c-afde-498b-8058-033c08cc0dbe', metadata={'source': './tax.docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015.

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [9]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\inflearn-rag-notebook-main\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [11]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [12]:
ai_message = qa_chain({"query": query})

C:\Users\le-ga\AppData\Local\Temp\ipykernel_33480\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [13]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [14]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 정확한 소득세액을 알기 위해서는 다양한 세액 공제 항목을 고려해야 합니다. 그러나 제공된 정보만으로는 정확한 소득세액을 계산할 수 없습니다. 일반적으로 연말정산 시 각종 소득공제와 세액공제를 적용하여 실제 납부할 소득세를 산출합니다. 따라서 정확한 소득세액을 알기 위해서는 국세청 홈택스 등을 통해 연말정산을 진행하거나 세무 전문가의 도움을 받는 것이 좋습니다.'}